<a href="https://colab.research.google.com/github/vishalrk1/pytorch/blob/main/Transfer_learning_and_fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dwonloading pizza and steak dataset

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# importing helper function
from helper_functions import create_tensorboard_callback, plot_loss_curves, pred_and_plot, unzip_data, walk_through_dir

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip

unzip_data('pizza_steak.zip')

--2021-12-06 08:07:58--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-12-06 08:07:59 (79.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]

--2021-12-06 08:08:02--  https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.8.128, 64.233.189.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.8.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109540975 (104M)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision

from torchvision import datasets, transforms, models

from torch.utils.data import SubsetRandomSampler

In [3]:
train_dir = '/content/pizza_steak/train'
test_dir = '/content/pizza_steak/test'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

num_workers = 0

BATCH_SIZE = 32

valid_size = 0.2

normalize = transforms.Normalize(
    [0.485, 0.456, 0.406], 
    [0.229, 0.224, 0.225]
)

train_transform = transforms.Compose([
    transforms.Resize((255, 255)),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    normalize,
])

test_transform = transforms.Compose([
    transforms.Resize((255, 255)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

# choose the training and test datasets
train_data = datasets.ImageFolder(train_dir, transform=train_transform)
test_data = datasets.ImageFolder(test_dir, transform=test_transform)

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=num_workers)
valid_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, sampler=valid_sampler, num_workers=num_workers)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, num_workers=num_workers)

cuda


In [5]:
classes = train_data.classes
class_len = len(classes)

classes

['pizza', 'steak']

In [29]:
# defining model
model = models.alexnet(pretrained=True)

for param in model.parameters():
  param.requires_grad = False

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [30]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [31]:
model.classifier[6] = nn.Linear(4096, 2)

for param in model.classifier[6].parameters():
  param.requires_grad = True

model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [32]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.001)
# lr shedular
exp_lr_scheduler = lr_scheduler.StepLR(optimizer=optimizer, step_size=5, gamma=0.1)

In [33]:
# training model
epochs = 10

results = {
    'train_losses': [],
    'valid_losses': [],
    'train_acc': [],
    'valid_acc': [],
}

for epoch in range(epochs):
  train_loss = 0
  valid_loss = 0

  train_acc = 0
  valid_acc = 0

  # training model
  model.train()
  for images, labels in train_dataloader:
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()

    output = model.forward(images)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()

    output_class = torch.softmax(output, dim=1).argmax(dim=1)
    train_acc += (output_class == labels).sum().item() / len(output) 

    exp_lr_scheduler.step()

  # validating model 
  model.eval()
  for images, labels in valid_dataloader:
      images, labels = images.to(device), labels.to(device)

      output = model(images)
      loss = criterion(output, labels)

      valid_loss += loss.item()

      output_class = torch.softmax(output, dim=1).argmax(dim=1)
      valid_acc += (output_class == labels).sum().item() / len(output)

  # Calculating losses
  train_loss = train_loss / len(train_dataloader)
  valid_loss = valid_loss / len(valid_dataloader)

  # Calculating accuracy
  train_acc = train_acc / len(train_dataloader)
  valid_acc = valid_acc / len(valid_dataloader)

  # appending results
  results['train_losses'].append(train_loss)
  results['valid_losses'].append(valid_loss)
  results['train_acc'].append(train_acc)
  results['valid_acc'].append(valid_acc)

  print(f'Epoch: {epoch+1}')
  print('Training Loss: {:.6f} \tTraining Accuracy: {:.2f}'.format(train_loss, train_acc))
  print('Validation Loss: {:.6f} \tValidation Accuracy: {:.2f}'.format(valid_loss, valid_acc))

Epoch: 1
Training Loss: 0.314599 	Training Accuracy: 0.86
Validation Loss: 0.254997 	Validation Accuracy: 0.89
Epoch: 2
Training Loss: 0.285580 	Training Accuracy: 0.88
Validation Loss: 0.233331 	Validation Accuracy: 0.92
Epoch: 3
Training Loss: 0.258993 	Training Accuracy: 0.90
Validation Loss: 0.241066 	Validation Accuracy: 0.91
Epoch: 4
Training Loss: 0.267278 	Training Accuracy: 0.89
Validation Loss: 0.249145 	Validation Accuracy: 0.89
Epoch: 5
Training Loss: 0.276104 	Training Accuracy: 0.90
Validation Loss: 0.241037 	Validation Accuracy: 0.91
Epoch: 6
Training Loss: 0.290476 	Training Accuracy: 0.88
Validation Loss: 0.235390 	Validation Accuracy: 0.91
Epoch: 7
Training Loss: 0.274121 	Training Accuracy: 0.88
Validation Loss: 0.232677 	Validation Accuracy: 0.91
Epoch: 8
Training Loss: 0.277908 	Training Accuracy: 0.89
Validation Loss: 0.262580 	Validation Accuracy: 0.89
Epoch: 9
Training Loss: 0.282804 	Training Accuracy: 0.88
Validation Loss: 0.247299 	Validation Accuracy: 0.89
E

In [34]:
# making predictions 

model.eval()
test_acc = 0 
test_loss = 0
for images, labels in test_dataloader:
  images, labels = images.to(device), labels.to(device)

  output = model(images)
  loss = criterion(output, labels)

  test_loss += loss.item()

  output_class = torch.softmax(output, dim=1).argmax(dim=1)

  test_acc += (output_class == labels).sum().item() / len(output) 

test_acc = test_acc / len(test_dataloader)
print(f"Test Accuracy: {test_acc}")

Test Accuracy: 0.963671875
